In [1]:
from pyscf import scf,gto
import numpy as np,numpy
from FcMole import FcM
from alch_deriv import alch_deriv

In [2]:
def DeltaV(mol,dL): # dL=[[atmidxs],[atmFcs]]
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV

In [3]:
#coefficients accurate dL^-8
d1=np.asarray([1/280,-4/105,1/5,-4/5,0,4/5,-1/5,4/105,-1/280])
d2=np.asarray([-1/560,8/315,-1/5,8/5,-205/72,8/5,-1/5,8/315,-1/560])
d3=np.asarray([-7/240,3/10,-169/120,61/30,0,-61/30,169/120,-3/10,7/240])
d4=np.asarray([7/240,-2/5,169/60,-122/15,91/8,-122/15,169/60,-2/5,7/240])
d5=np.asarray([1/6,-3/2,13/3,-29/6,0,29/6,-13/3,3/2,-1/6])
d6=np.asarray([-1/4,3,-13,29,-75/2,29,-13,3,-1/4])

In [4]:
dl=.05
Ps=[]
Es=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
    mf=scf.RHF(fm)
    Es.append(mf.scf(dm0=mf.init_guess_by_atom()))
    Ps.append(mf.make_rdm1())

prima opzione
converged SCF energy = -112.742145890619
prima opzione
converged SCF energy = -112.351169769144
prima opzione
converged SCF energy = -111.967431228079
prima opzione
converged SCF energy = -111.590993296104
prima opzione
converged SCF energy = -111.221914650137
prima opzione
converged SCF energy = -110.860249698719
prima opzione
converged SCF energy = -110.506048710203
prima opzione
converged SCF energy = -110.15935797469
prima opzione
converged SCF energy = -109.820219990832


In [5]:
coeffs=np.array([d1,d2,d3,d4,d5,d6])
coeffs[5]

array([ -0.25,   3.  , -13.  ,  29.  , -37.5 ,  29.  , -13.  ,   3.  ,
        -0.25])

In [6]:
mol0=gto.M(atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mf0=scf.RHF(mol0)
E0=mf0.scf()
P0=mf0.make_rdm1()

converged SCF energy = -111.221914650137


In [7]:
mf0.energy_nuc()

22.857142857142854

In [8]:
molT=FcM(fcs=[1,-1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mfT=scf.RHF(molT)
ET=mfT.scf(dm0=mf.init_guess_by_atom())
PT=mfT.make_rdm1()

prima opzione
converged SCF energy = -105.444757226604


In [9]:
#dE/dZ^(n+1) = \int dP/dZ^n * dV
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e)
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

-104.39049415776398
-105.39711356625223 -1.006619408488245
-105.4668732355181 -0.06975966926586609
-105.43921627219379 0.02765696332430982
-105.44261387912347 -0.0033976069296815154
-105.44451694630392 -0.0019030671804487417
-105.44703456252344 -0.0025176162195264693


In [10]:
#expanding energies
e=E0
print(e)
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1))

-111.22191465013731
-103.9143043129372 7.307610337200099
-105.39711229992677 -1.4828079869895703
-105.46684776481446 -0.06973546488768059
-105.43920946755475 0.027638297259713337
-105.44316650051383 -0.003957032959078787
-105.4461164494212 -0.002949948907371435


In [38]:
d1,d2,d3= alch_deriv(mf0, dL=[[0,1],[1,-1]])

In [31]:
dl=.05
d3s=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_atom())
    d1,d2,d3= alch_deriv(mf, dL=[[0,1],[1,-1]])
    d3s.append(d3)

prima opzione
converged SCF energy = -112.742145890619
prima opzione
converged SCF energy = -112.351169769144
prima opzione
converged SCF energy = -111.967431228079
prima opzione
converged SCF energy = -111.590993296104
prima opzione
converged SCF energy = -111.221914650137
prima opzione
converged SCF energy = -110.860249698719
prima opzione
converged SCF energy = -110.506048710203
prima opzione
converged SCF energy = -110.15935797469
prima opzione
converged SCF energy = -109.820219990832


In [32]:
d3s=np.asarray(d3s)
d3s

array([0.55673685, 0.52175295, 0.4867074 , 0.45212503, 0.41841326,
       0.38588167, 0.35476061, 0.32521817, 0.29737508])

In [59]:
e=E0+(mfT.energy_nuc()-mf0.energy_nuc())
print(e,"       ",e-ET)
e+=d1
print(e,d1,e-ET)
e+=d2/2
print(e,d2/2,e-ET)
e+=d3/6
print(e,d3/6,e-ET)
for i in range (6):
    e+=np.sum(coeffs[i]*d3s)/np.math.factorial(i+4)/(-dl)**(i+1)
    print("APDFT{}".format(i+4),e,np.sum(coeffs[i]*d3s)/np.math.factorial(i+4)/(-dl)**(i+1),e-ET)

-112.65048607870874         7.267471048569803
-119.00571609489158 -6.355230016182839 0.9122410323869588
-120.01233348044484 -1.0066173855532505 -0.09437635316629667
-119.94259794894512 0.06973553149971197 -0.024640821666579882
APDFT4 -119.91495955191239 0.02763839703273769 0.00299757536615175
APDFT5 -119.9110036490729 0.003955902839495371 0.0069534782056450695
APDFT6 -119.91395346219306 -0.0029498131201685027 0.004003665085477337
APDFT7 -119.91646010168465 -0.0025066394915791816 0.0014970255938919763
APDFT8 -119.91788708499166 -0.001426983307008487 7.00422868789019e-05
APDFT9 -119.91818887871074 -0.000301793719069231 -0.00023175143219589245


In [61]:
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e,np.einsum('ij,ij',P0,dV))
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(-dl)**(i+1))    # -dl to invert the direction
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print("APDFT{}".format(i+2),e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2),e-ET)

-119.00571609489158 -6.355230016182839
APDFT2 -120.01233550337983 -1.006619408488245 -0.09437837610128952
APDFT3 -119.94257583411397 0.06975966926586609 -0.024618706835425996
APDFT4 -119.91491887078966 0.02765696332430982 0.0030382564888782326
APDFT5 -119.91152126385998 0.0033976069296815154 0.0064358634185595065
APDFT6 -119.91342433104043 -0.0019030671804487417 0.004532796238109427
APDFT7 -119.91090671482091 0.0025176162195264693 0.007050412457630273


In [52]:
ET

-119.91795712727854